In [1]:
from rdkit import Chem
from rdkit.Chem import rdmolops
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

#nx approach derived from https://github.com/maxhodak/keras-molecules/pull/32/commits

In [4]:

# Create a function called "chunks" with two arguments, l and n:
def chunker(l1, l2, chunk):
    chuncks = []
    for i in range(len(l2)-len(l1)):
        chuncks.append(l2[i:i+chunk])
    return chuncks
        
    
    
    #levenshtein distance
    #https://en.wikibooks.org/wiki/Algorithm_Implementation/Strings/Levenshtein_distance#Python
def levenshtein(s1, s2, cap):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)
    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1] < cap
def proximity(l1, l2, mismatches_cap, fine_cap = 0):
    mismatches = 0
    fine_mismatches = 0 
    if len(l1) == len(l2):
        for i in range(len(l1)):
            if l1[i] != l2[i] and fine_cap == 0:
                mismatches+=1
            elif l1[i] != l2[i]:
                if not levenshtein(l1, l2, fine_cap):
                    mismatches+=1
        if mismatches >= mismatches_cap:
            return False
        else:
            return True
    elif len(l1) < len(l2):
        if fine_cap == 0:
            for i in range(0, mismatches_cap):
                if l1[i:] in l2:
                    return True
                if l1[:i] in l2:
                    return True
            return False
        else:
            for i in range(0, mismatches_cap):
                for j in chunker(l1, l2, i):
                    if levenshtein(l1, j, fine_cap):
                        return True
                        
    
        
    else:
        if fine_cap == 0:
            for i in range(0,mismatches_cap):
                if l2[i:] in l1:
                    return True
                if l2[:i] in l1:
                    return True
            return False
        else:
            for i in range(0, mismatches_cap):
                for j in chunker(l2, l1, i):
                    if levenshtein(l2, j, fine_cap):
                        return True
                        

        
        
class Mol_Graph:
    def __init__(self, mol_str):
        self.G = nx.Graph()
        self.mol = Chem.MolFromSmiles(mol_str)
        self.node_attributes =["atomic_num","symbol", "formal_charge", "chiral_tag", "hybridization", "num_explicit_hs","is_aromatic"]
        self.edge_attributes = ["bond_type"]
        self.paths = []
        self.attribute_paths = []
        
        for atom in self.mol.GetAtoms():
            self.G.add_node(atom.GetIdx(),
                            symbol=atom.GetSymbol(),
                       atomic_num=atom.GetAtomicNum(),
                       formal_charge=atom.GetFormalCharge(),
                       chiral_tag=atom.GetChiralTag(),
                       hybridization=atom.GetHybridization(),
                       num_explicit_hs=atom.GetNumExplicitHs(),
                       is_aromatic=atom.GetIsAromatic())
            
        for bond in self.mol.GetBonds():
            self.G.add_edge(bond.GetBeginAtomIdx(),
                       bond.GetEndAtomIdx(),
                       bond_type=bond.GetBondType())
    def draw_graph(self):
        plt.subplot(121)
        nx.draw(self.G, with_labels=True, font_weight='bold')
        plt.show()
        
    def generate_all_simple_paths(self, min_path_length=2):
        #all_simple_paths(G, source, target[, cutoff])	Generate all simple paths in the graph G from source to target.
        for n in self.G.nodes():
            if n+min_path_length <= len(self.G.nodes()):
                for j in range(n+min_path_length, len(self.G.nodes())):
                    self.paths.append(list(nx.all_simple_paths(self.G, n, j))) 
    def generate_attribute_paths(self, attributes=["symbol", "is_aromatic", "hybridization"]):
        if attributes==0:
            attributes=[nx.get_node_attributes(self.G, a) for a in self.node_attributes]
        else:
            attributes=[nx.get_node_attributes(self.G, a) for a in attributes]
        for i in self.paths:
            for j in i:
                attribute_path = []
                for z in j:
                    attribute_path.append([a[z] for a in attributes])
                self.attribute_paths.append(attribute_path)
    
                
    def find_matching_attribute_path(self, sub_paths, returned =0, min_path_length=2):
        matches = []
        perfect_matches = []
        for i in sub_paths:
            if len(i) > 2:
                for j in self.attribute_paths:
                    if len(j) >2:
                        if i == j:
                            perfect_matches.append(j)
                            matches.append(j)
                        if i in j:
                            matches.append(j)
        if returned == 0:
            return matches, perfect_matches
        elif returned == 1:
            return matches
        elif returned == 2:
            return perfect_matches

    def find_close_attribute_paths(self, sub_paths, mismatch_cap=2, min_path_length=2, fine_cap = 0):
        close_paths = []
        for i in sub_paths:
            if len(i)>min_path_length:
                for j in self.attribute_paths:
                    if len(j)>min_path_length:
                        if proximity(i, j, mismatch_cap, fine_cap=fine_cap) == True:
                            close_paths.append(j)
        return close_paths
                    
        
t_mol = "CCOC(=O)C1=C[C@@H](OC(CC)CC)[C@H](NC(C)=O)[C@@H](N)C1"   
s_mol = "CSOC(=O)"
g = Mol_Graph(t_mol)
g.generate_all_simple_paths()
g.generate_attribute_paths()
s = Mol_Graph(s_mol)
s.generate_all_simple_paths()
s.generate_attribute_paths()
apaths = g.find_matching_attribute_path(s.attribute_paths, returned=1)
cpaths = g.find_close_attribute_paths(s.attribute_paths)

for i in apaths:
    for j in cpaths:
        if i != j:
            print(i, j)
# Proximity searches and equivelences? 

[['O', False, rdkit.Chem.rdchem.HybridizationType.SP2], ['C', False, rdkit.Chem.rdchem.HybridizationType.SP2], ['O', False, rdkit.Chem.rdchem.HybridizationType.SP2]] [['C', False, rdkit.Chem.rdchem.HybridizationType.SP3], ['C', False, rdkit.Chem.rdchem.HybridizationType.SP3], ['O', False, rdkit.Chem.rdchem.HybridizationType.SP2]]
[['O', False, rdkit.Chem.rdchem.HybridizationType.SP2], ['C', False, rdkit.Chem.rdchem.HybridizationType.SP2], ['O', False, rdkit.Chem.rdchem.HybridizationType.SP2]] [['C', False, rdkit.Chem.rdchem.HybridizationType.SP3], ['C', False, rdkit.Chem.rdchem.HybridizationType.SP3], ['O', False, rdkit.Chem.rdchem.HybridizationType.SP2], ['C', False, rdkit.Chem.rdchem.HybridizationType.SP2]]
[['O', False, rdkit.Chem.rdchem.HybridizationType.SP2], ['C', False, rdkit.Chem.rdchem.HybridizationType.SP2], ['O', False, rdkit.Chem.rdchem.HybridizationType.SP2]] [['C', False, rdkit.Chem.rdchem.HybridizationType.SP3], ['C', False, rdkit.Chem.rdchem.HybridizationType.SP3], ['O'